In [1]:
import glacierml as gl
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_column',None)

2024-04-22 08:22:11.133711: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-22 08:22:11.133734: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
[
    data_path,RGI_path, glathida_path, 
    coregistration_testing_path, 
    arch_test_path, LOO_path
] = gl.set_paths(home_path = '/home/simonhans/glacierml/')

In [3]:
glathida = pd.read_csv(os.path.join(glathida_path, 'T.csv'))
glathida = glathida.dropna(subset = ['MEAN_THICKNESS'])
glathida = glathida.reset_index().drop('index',axis = 1)

In [4]:
if not os.path.exists(
    os.path.join(
        data_path, 'matched.pkl'
)):
    df = gl.match_GlaThiDa_RGI_index(
    RGI,glathida,verbose = True, useMP = True
)
    df = pd.concat(
        [
            glathida,
            pd.Series(RGI_ids,name = 'RGIId'),
            pd.Series(centroid_distances,name = 'centroid distance')
        ],axis = 1
    )
    df.to_pickle(os.path.join(data_path,'matched.pkl'))
    
else:
    df = pd.read_pickle(os.path.join(data_path,'matched.pkl'))    

In [5]:
def load_training_data(data_path):  
    # Read matched dataset and select data
    df = pd.read_pickle(os.path.join(data_path,'matched.pkl'))
    df = df[[
        'RGIId','MEAN_THICKNESS','RGI Centroid Distance','SURVEY_DATE',
        'AREA','MEAN_THICKNESS_UNCERTAINTY','GLACIER_NAME'
    ]]
    df = df.rename(columns = {
        'MEAN_THICKNESS':'Thickness',
    })
    
    # Load RGI to get RGI attributes
    RGI_path = os.path.join(data_path,'RGI')
    RGI = gl.load_RGI(RGI_path)

    # Put the data together
    df = pd.merge(df,RGI,how = 'inner', on = 'RGIId')
    
    return df

In [6]:
def coregister_data(
    data_path,
    coregistration = '1',
):
    
    df = load_training_data(data_path)
    
    
    import configparser
    config = configparser.ConfigParser()
    config.read('model_coregistration.txt')   
    
    
    df['RGI Centroid Distance'] = df['RGI Centroid Distance'].astype(str).str[:-2].astype(float)
    df['RGI Centroid Distance'] = df['RGI Centroid Distance'] * 1e3
    
    rad = pd.Series(np.sqrt((df['Area']*1e6) / np.pi),name = 'AVG Radius')
    
    df = pd.concat([df,rad],axis = 1)
    
    distance_test = pd.Series(
        df['RGI Centroid Distance'] / df['AVG Radius'],
        name = 'distance test'
    )
    
    
    df = pd.concat([df,distance_test],axis = 1)
    
    df = df.drop(
        df[df['distance test'] >= float(config[coregistration]['distance threshold'])].index
    )
    
    # Look at size difference between data
    ps = pd.Series(
        (df['Area'] - df['AREA']) / df['AREA'],
        name = 'perc difference'
    )
    df = pd.concat([df,ps],axis = 1)
    area_scrubber = config[coregistration]['area scrubber']
    if area_scrubber == 'on':
        size_threshold = float(config[coregistration]['size threshold'])
        df = df[
            (df['perc difference'] <= size_threshold) & 
            (df['perc difference'] >= -size_threshold)
        ]
    
    df = df.drop(df[df['RGIId'].duplicated(keep = False)].index)

    
    
    return df


In [7]:
df = coregister_data(data_path,'4')

df

,RGIId,Thickness,RGI Centroid Distance,SURVEY_DATE,AREA,MEAN_THICKNESS_UNCERTAINTY,GLACIER_NAME,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Status,Connect,Form,TermType,Surging,Linkages,Name,region,AVG Radius,distance test,perc difference
0,RGI60-08.00251,72.0,167.899602,19790399.0,1.30000,NaN,ISFALLSGLAC,G018564E67915N,20020731,-9999999,18.564000,67.915000,8,1,1.438,1204,1698,1526,16.9,98,1941,0,0,0,0,9,0,Isfallsglaciaeren,08,676.557179,0.248168,0.106154
1,RGI60-08.00188,84.0,0.000000,19790399.0,4.10000,NaN,RABOTS GLACIAER,G018496E67910N,20020731,-9999999,18.496000,67.910000,8,1,3.696,1207,2070,1443,15.9,282,3954,0,0,0,0,9,1,Rabots Glaciaer,08,1084.653557,0.000000,-0.098537
2,RGI60-08.00213,99.0,337.228663,19790399.0,3.10000,NaN,STORGLACIAEREN,G018569E67903N,20020731,-9999999,18.569000,67.903000,8,1,3.405,1143,1797,1414,13.2,102,3636,0,0,0,0,9,1,Storglaciaeren,08,1041.078845,0.323922,0.098387
3,RGI60-02.18778,99.0,0.000000,19759999.0,2.00000,NaN,SOUTH CASCADE,G238943E48357N,19589999,-9999999,-121.057350,48.356980,2,4,2.924,1613,2196,1891,12.8,350,3338,0,0,0,0,0,1,South Cascade Glacier WA,02,964.747691,0.000000,0.462000
6,RGI60-02.16499,20.0,0.000000,19819999.0,1.38000,NaN,WHITNEY,G237789E41414N,19809999,-9999999,-122.211330,41.414010,2,4,1.291,3033,4180,3492,21.5,332,3168,0,0,0,0,0,9,Whitney Glacier CA,02,641.044509,0.000000,-0.064493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,RGI60-13.08624,47.0,0.024938,20130828.0,2.61000,2.0,TSENTRALNIY TUYUKSU,G077081E43044N,19990909,-9999999,77.080587,43.043653,13,3,2.838,3427,4185,3764,19.1,359,3131,0,0,0,0,9,1,Tsentralniy Tuyuksu Glacier,13,950.454342,0.000026,0.087356
495,RGI60-16.01655,23.0,285.551025,20150916.0,0.52568,1.0,NORTHERN ICE FIELD,G037351E03057S,20000221,-9999999,37.350700,-3.057150,16,3,0.616,5663,5794,5771,9.9,355,905,0,0,0,0,0,9,TZ3D500A3001 Northern Ice,16,442.807961,0.644864,0.171816
496,RGI60-07.00209,27.0,0.000000,20070417.0,0.37000,2.0,ARIEBREEN,G015485E77027N,20080901,-9999999,15.485400,77.027200,7,1,0.418,323,587,422,18.8,144,1084,0,0,0,0,0,9,Areibreen,07,364.765037,0.000000,0.129730
497,RGI60-07.00496,76.0,0.000000,20100424.0,4.60000,9.0,AUSTRE LOVENBREEN,G012161E78871N,20070901,-9999999,12.161400,78.871000,7,1,5.016,123,684,388,13.8,350,4111,0,0,0,0,0,9,Austre Lovenbreen,07,1263.583155,0.000000,0.090435
